In [1]:
### IDS RISK MAPPING

In [2]:
# Import Libraries
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

from sklearn.model_selection import train_test_split, GroupKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, 
    roc_auc_score, roc_curve, confusion_matrix, classification_report
)
from sklearn.metrics import roc_curve, auc
from scipy.interpolate import PchipInterpolator
from sklearn.calibration import calibration_curve
from sklearn.cluster import KMeans
from sklearn.base import clone
from statsmodels.stats.outliers_influence import variance_inflation_factor

import pickle
from collections import defaultdict

np.random.seed(42)

print("Imported Successfully")

Imported Successfully


In [3]:
# STEP 1: Load Study Area

grid_path = "D:/MURP/Thesis/Thesis/New Analysis/FN.shp"
grid = gpd.read_file(grid_path)
print(f"Grid cells loaded: {len(grid):,}")
print(f"Grid CRS: {grid.crs}")

centroids = grid.geometry.centroid
grid['centroid_x'] = centroids.x
grid['centroid_y'] = centroids.y

print("Study Area Loaded")

Grid cells loaded: 454,558
Grid CRS: EPSG:32645
Study Area Loaded


In [4]:
# STEP 2: Extract Raster Values

raster_paths = {
    'population_density': "D:/MURP/IDS Review/Raster Data/Population_Density.tif",
    'dem': "D:/MURP/IDS Review/Raster Data/DEM.tif",
    'wgr': "D:/MURP/IDS Review/Raster Data/WGR.tif",
    'poverty_index': "D:/MURP/IDS Review/Raster Data/Poverty_Index.tif",
    'lst': "D:/MURP/IDS Review/Raster Data/LST.tif",
    'drainage': "D:/MURP/IDS Review/Raster Data/Drain.tif",
    'buildings': "D:/MURP/IDS Review/Raster Data/Building.tif",
    'railine': "D:/MURP/IDS Review/Raster Data/Rail.tif",
    'road': "D:/MURP/IDS Review/Raster Data/Road.tif",
    'road_intersection': "D:/MURP/IDS Review/Raster Data/Intersection.tif",
    'waterbody': "D:/MURP/IDS Review/Raster Data/Waterbody.tif",
    'sts': "D:/MURP/IDS Review/Raster Data/STS.tif"
}

NODATA_SENTINEL = -3.4028234663852886e+38

for name, path in raster_paths.items():
    with rasterio.open(path) as src:
        raster_crs = src.crs
        
        if grid.crs != raster_crs:
            grid_proj = grid.to_crs(raster_crs)
            pts = grid_proj.geometry.centroid
        else:
            pts = centroids
        
        coords = [(pt.x, pt.y) for pt in pts]
        vals = [v[0] for v in src.sample(coords)]
        
        vals = [
            np.nan if (v == src.nodata or v == NODATA_SENTINEL or np.isclose(v, NODATA_SENTINEL))
            else v for v in vals
        ]
        
        grid[name] = vals
    
    print(f"  Extracted: {name}")

  Extracted: population_density
  Extracted: dem
  Extracted: wgr
  Extracted: poverty_index
  Extracted: lst
  Extracted: drainage
  Extracted: buildings
  Extracted: railine
  Extracted: road
  Extracted: road_intersection
  Extracted: waterbody
  Extracted: sts


In [5]:
# STEP 3: Imputation of Missing Values

data = grid.drop(columns=['geometry']).copy()
data['grid_idx'] = grid.index

missing_before = data.isnull().sum().sum()
print(f"Missing values before imputation: {missing_before:,}")

if missing_before > 0:
    numeric_cols = data.select_dtypes(include=[np.number]).columns.tolist()
    cols_to_impute = [col for col in numeric_cols if data[col].isnull().any() and col != 'grid_idx']
    
    idx_to_row = {grid_idx: i for i, grid_idx in enumerate(data['grid_idx'])}
    
    imputed_count = 0
    for row_idx, row in data.iterrows():
        if row.isnull().any():
            grid_idx = row['grid_idx']
            cell_geom = grid.loc[grid_idx, 'geometry']
            neighbors = grid[grid.geometry.touches(cell_geom)]
            neighbor_grid_indices = neighbors.index.tolist()
            neighbor_data_indices = [idx_to_row[gi] for gi in neighbor_grid_indices if gi in idx_to_row]
            
            for col in cols_to_impute:
                if pd.isnull(row[col]):
                    neighbor_values = data.iloc[neighbor_data_indices][col].dropna()
                    if len(neighbor_values) > 0:
                        data.at[row_idx, col] = neighbor_values.mean()
                        imputed_count += 1
    
    print(f"  Spatially imputed: {imputed_count:,} values")
    
    remaining_missing = data.isnull().sum().sum()
    if remaining_missing > 0:
        for col in cols_to_impute:
            if data[col].isnull().any():
                data[col].fillna(data[col].mean(), inplace=True)
        print(f"  Filled remaining with column means")

print(f"Missing values after imputation: {data.isnull().sum().sum()}")

Missing values before imputation: 58,424
  Spatially imputed: 57,441 values
  Filled remaining with column means
Missing values after imputation: 0


In [11]:
# STEP 4: Data Partitioning

data['label'] = (data['Status'] == 'Positive').astype(int)

P_indices = data[data['label'] == 1].index.tolist()
U_indices = data[data['label'] == 0].index.tolist()

n_P = len(P_indices)
n_U = len(U_indices)

print(f"Positive samples (P): {n_P:,}")
print(f"Unlabeled samples (U): {n_U:,}")
print(f"Class imbalance ratio (U:P): {n_U/n_P:.1f}:1")

if n_P < 10:
    raise ValueError(f"Insufficient positive samples: {n_P}")

P_train_indices, P_test_indices = train_test_split(
    P_indices, test_size=0.3, random_state=42, shuffle=True
)

n_P_train = len(P_train_indices)
n_P_test = len(P_test_indices)

print(f"\nPositive split (no leakage):")
print(f"  Training positives: {n_P_train}")
print(f"  Test positives: {n_P_test}")

U_available_for_test = list(set(U_indices) - set(P_train_indices) - set(P_test_indices))
U_test_indices = np.random.choice(U_available_for_test, size=n_P_test, replace=False).tolist()

print(f"\nTest set composition:")
print(f"  Test positives: {n_P_test}")
print(f"  Test unlabeled: {len(U_test_indices)}")

Positive samples (P): 342
Unlabeled samples (U): 454,216
Class imbalance ratio (U:P): 1328.1:1

Positive split (no leakage):
  Training positives: 239
  Test positives: 103

Test set composition:
  Test positives: 103
  Test unlabeled: 103


In [12]:
# STEP 5: Predictor Features

exclude_cols = ['Status', 'label', 'ID', 'grid_idx', 'FID', 'centroid_x', 'centroid_y']
predictor_cols = [col for col in data.columns if col not in exclude_cols]

distance_based_features = [
    'road', 'railine', 'drainage', 'road_intersection', 
    'buildings', 'sts', 'waterbody'
]

non_distance_features = [col for col in predictor_cols if col not in distance_based_features]

print(f"Total predictor features: {len(predictor_cols)}")
print(f"  Distance-based: {len(distance_based_features)}")
print(f"  Non-distance: {len(non_distance_features)}")

Total predictor features: 12
  Distance-based: 7
  Non-distance: 5


In [13]:
# STEP 6: Predictor for Tree vs Linear/Kernel-Based Models

data_tree = data[predictor_cols].copy()

data_linear = data[predictor_cols].copy()
for col in distance_based_features:
    data_linear[col] = np.log1p(data_linear[col])

print("Feature transformations:")
print("  Tree models: raw distances")
print("  Linear/Kernel-based: log-transformed distances")

scaler_tree = StandardScaler()
X_tree_full = scaler_tree.fit_transform(data_tree.values)

scaler_linear = StandardScaler()
X_linear_full = scaler_linear.fit_transform(data_linear.values)

y_full = data['label'].values

print(f"Feature matrix: {X_tree_full.shape}")

Feature transformations:
  Tree models: raw distances
  Linear/Kernel-based: log-transformed distances
Feature matrix: (454558, 12)


In [14]:
# STEP 7: VIF Check

X_linear_df = pd.DataFrame(X_linear_full, columns=predictor_cols)

def compute_vif(df):
    vif_data = pd.DataFrame()
    vif_data['Feature'] = df.columns
    vif_data['VIF'] = [variance_inflation_factor(df.values, i) for i in range(df.shape[1])]
    return vif_data.sort_values('VIF', ascending=False)

vif = compute_vif(X_linear_df)
print("\nVIF values:")
print(vif)


VIF values:
               Feature       VIF
4                  lst  2.736554
2                  wgr  2.634594
3        poverty_index  2.489704
8                 road  2.265788
9    road_intersection  2.248761
5             drainage  1.980847
11                 sts  1.563864
0   population_density  1.553409
6            buildings  1.552092
1                  dem  1.386708
7              railine  1.334183
10           waterbody  1.069423


In [15]:
# STEP 8: Spatial Blocks

coords = data[['centroid_x', 'centroid_y']].values
n_blocks = 5
kmeans = KMeans(n_clusters=n_blocks, random_state=42, n_init=10)
spatial_blocks = kmeans.fit_predict(coords)

data['spatial_block'] = spatial_blocks

print(f"Created {n_blocks} spatial blocks")
for block_id in range(n_blocks):
    block_size = np.sum(spatial_blocks == block_id)
    print(f"  Block {block_id}: {block_size:,} cells ({100*block_size/len(data):.1f}%)")

print("\nPositive distribution across spatial blocks:")
for block_id in range(n_blocks):
    block_mask = spatial_blocks == block_id
    n_positives_in_block = np.sum((data['label'] == 1) & block_mask)
    print(f"  Block {block_id}: {n_positives_in_block} positives")

Created 5 spatial blocks
  Block 0: 111,250 cells (24.5%)
  Block 1: 70,505 cells (15.5%)
  Block 2: 74,662 cells (16.4%)
  Block 3: 99,911 cells (22.0%)
  Block 4: 98,230 cells (21.6%)

Positive distribution across spatial blocks:
  Block 0: 83 positives
  Block 1: 52 positives
  Block 2: 29 positives
  Block 3: 76 positives
  Block 4: 102 positives


In [16]:
# STEP 9: Balanced Training Dataset

n_sample_U = min(n_P_train, len([i for i in U_indices if i not in P_test_indices and i not in U_test_indices]))
U_available_for_tuning = [i for i in U_indices if i not in P_test_indices and i not in U_test_indices]
sampled_U_tuning = np.random.choice(U_available_for_tuning, size=n_sample_U, replace=False).tolist()

balanced_indices = P_train_indices + sampled_U_tuning
balanced_labels = [1]*n_P_train + [0]*n_sample_U

X_tree_balanced = X_tree_full[balanced_indices]
X_linear_balanced = X_linear_full[balanced_indices]

y_balanced = np.array(balanced_labels)
groups_balanced = spatial_blocks[balanced_indices]

print(f"Balanced dataset size: {len(balanced_indices):,}")
print(f"  Positives: {n_P_train}")
print(f"  Unlabeled: {n_sample_U}")

Balanced dataset size: 478
  Positives: 239
  Unlabeled: 239


In [17]:
# STEP 10: Hyperparameter tuning - Tree-Based Models

tree_models = {
    'RandomForest': {
        'model': RandomForestClassifier(random_state=42, n_jobs=-1, class_weight='balanced'),
        'params': {
            'n_estimators': [200, 500],
            'max_depth': [10, 20, None],
            'min_samples_split': [5, 10],
            'min_samples_leaf': [2, 4],
            'max_features': ['sqrt', 'log2']
        }
    },
    'XGBoost': {
        'model': XGBClassifier(random_state=42, n_jobs=-1, tree_method='hist', eval_metric='logloss'),
        'params': {
            'n_estimators': [100, 300],
            'max_depth': [3, 5, 7],
            'learning_rate': [0.01, 0.05, 0.1],
            'subsample': [0.8, 1.0],
            'colsample_bytree': [0.8, 1.0],
            'min_child_weight': [1, 3]
        }
    },
    'LightGBM': {
        'model': LGBMClassifier(random_state=42, n_jobs=-1, class_weight='balanced', verbosity=-1),
        'params': {
            'n_estimators': [100, 300],
            'max_depth': [3, 5, 7],
            'learning_rate': [0.01, 0.05, 0.1],
            'num_leaves': [15, 31, 63],
            'min_child_samples': [10, 20],
            'subsample': [0.8, 1.0]
        }
    }
}

best_tree_models = {}
tree_tuning_results = []

n_splits = min(5, len(np.unique(groups_balanced)))
cv_splitter = GroupKFold(n_splits=n_splits)

for model_name, config in tree_models.items():
    print(f"\nTuning {model_name}...")
    
    grid_search = GridSearchCV(
        estimator=config['model'],
        param_grid=config['params'],
        cv=cv_splitter,
        scoring='roc_auc',
        n_jobs=-1,
        verbose=0
    )
    
    grid_search.fit(X_tree_balanced, y_balanced, groups=groups_balanced)
    
    best_tree_models[model_name] = grid_search.best_estimator_
    
    tree_tuning_results.append({
        'Model': model_name,
        'Best_Params': str(grid_search.best_params_),
        'CV_AUC_Mean': grid_search.best_score_,
        'CV_AUC_Std': grid_search.cv_results_['std_test_score'][grid_search.best_index_]
    })
    
    print(f"  Best CV AUC: {grid_search.best_score_:.4f} ±{grid_search.cv_results_['std_test_score'][grid_search.best_index_]:.4f}")


Tuning RandomForest...
  Best CV AUC: 0.8828 ±0.0272

Tuning XGBoost...
  Best CV AUC: 0.8769 ±0.0313

Tuning LightGBM...
  Best CV AUC: 0.8670 ±0.0271


In [18]:
# STEP 11: Hyperparameter tuning - Linear/Kernel models

X_linear_balanced_filtered = X_linear_full[balanced_indices]

linear_models = {
    'LogisticReg': {
        'model': LogisticRegression(random_state=42, max_iter=2000, n_jobs=-1, class_weight='balanced'),
        'params': {
            'C': [0.01, 0.1, 1, 10],
            'penalty': ['l2'],
            'solver': ['lbfgs']
        }
    },
    'SVM': {
        'model': SVC(random_state=42, probability=True, class_weight='balanced', cache_size=500),
        'params': {
            'C': [0.1, 1, 10, 100],
            'kernel': ['rbf', 'poly'],
            'gamma': ['scale', 0.01, 0.1]
        }
    },
    'MLP': {
        'model': MLPClassifier(random_state=42, max_iter=1000, early_stopping=True),
        'params': {
            'hidden_layer_sizes': [(100,), (100, 50), (150, 75, 25)],
            'activation': ['relu', 'tanh'],
            'alpha': [0.0001, 0.001, 0.01],
            'learning_rate': ['adaptive']
        }
    },
    'KNN': {
        'model': KNeighborsClassifier(n_jobs=-1),
        'params': {
            'n_neighbors': [5, 9, 15, 21],
            'weights': ['uniform', 'distance'],
            'metric': ['euclidean', 'manhattan'],
            'p': [1, 2]
        }
    }
}

best_linear_models = {}
linear_tuning_results = []

for model_name, config in linear_models.items():
    print(f"\nTuning {model_name}...")
    
    grid_search = GridSearchCV(
        estimator=config['model'],
        param_grid=config['params'],
        cv=cv_splitter,
        scoring='roc_auc',
        n_jobs=-1,
        verbose=0
    )
    
    grid_search.fit(X_linear_balanced_filtered, y_balanced, groups=groups_balanced)
    
    best_linear_models[model_name] = grid_search.best_estimator_
    
    linear_tuning_results.append({
        'Model': model_name,
        'Best_Params': str(grid_search.best_params_),
        'CV_AUC_Mean': grid_search.best_score_,
        'CV_AUC_Std': grid_search.cv_results_['std_test_score'][grid_search.best_index_]
    })
    
    print(f"  Best CV AUC: {grid_search.best_score_:.4f} ±{grid_search.cv_results_['std_test_score'][grid_search.best_index_]:.4f}")

best_models = {**best_tree_models, **best_linear_models}

tuning_df = pd.DataFrame(tree_tuning_results + linear_tuning_results)
tuning_df = tuning_df.sort_values('CV_AUC_Mean', ascending=False)

print(tuning_df[['Model', 'CV_AUC_Mean', 'CV_AUC_Std']].to_string(index=False))

tuning_df.to_csv('hyperparameter_tuning.csv', index=False)


Tuning LogisticReg...
  Best CV AUC: 0.8755 ±0.0124

Tuning SVM...
  Best CV AUC: 0.8775 ±0.0221

Tuning MLP...
  Best CV AUC: 0.8742 ±0.0169

Tuning KNN...
  Best CV AUC: 0.8625 ±0.0179
       Model  CV_AUC_Mean  CV_AUC_Std
RandomForest     0.882828    0.027185
         SVM     0.877490    0.022065
     XGBoost     0.876874    0.031328
 LogisticReg     0.875467    0.012441
         MLP     0.874249    0.016911
    LightGBM     0.867036    0.027077
         KNN     0.862473    0.017893


In [19]:
# STEP 12: PU Bagging

n_iterations = 500
print(f"Iterations per model: {n_iterations}")

sampled_U_indices_iterations = np.array([
    np.random.choice([i for i in U_indices if i not in P_test_indices and i not in U_test_indices], 
                     size=n_P_train, replace=False)
    for _ in range(n_iterations)
])

def calculate_ll_metric(tp, fp, fn, tn):
    if (tp + fp) == 0 or (tp + fn) == 0:
        return 0.0
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    if precision + recall == 0:
        return 0.0
    return 2 * precision * recall / (precision + recall)

reliability_thresholds = [0.55, 0.60, 0.65, 0.70]

threshold_sensitivity_results = defaultdict(list)
final_predictions_by_threshold = {}

for model_name in best_models.keys():
    print(f"PU BAGGING: {model_name}")
    
    is_tree_model = model_name in best_tree_models
    base_model = best_models[model_name]
    X_full_model = X_tree_full if is_tree_model else X_linear_full
    X_balanced_model = X_tree_balanced if is_tree_model else X_linear_balanced_filtered
    
    for threshold in reliability_thresholds:
        iteration_predictions = []
        iteration_weights = []
        valid_count = 0
        
        for i in range(n_iterations):
            try:
                Ui = sampled_U_indices_iterations[i]
                Mi_indices = np.concatenate([P_train_indices, Ui])
                Mi_labels = np.array([1]*n_P_train + [0]*n_P_train)
                
                X_Mi = X_full_model[Mi_indices]
                y_Mi = Mi_labels
                
                X_train_i, X_test_i, y_train_i, y_test_i = train_test_split(
                    X_Mi, y_Mi, test_size=0.2, random_state=i, stratify=y_Mi
                )
                
                model_i = clone(base_model)
                model_i.fit(X_train_i, y_train_i)
                
                y_pred_test = model_i.predict(X_test_i)
                cm = confusion_matrix(y_test_i, y_pred_test)
                
                if cm.shape != (2, 2):
                    continue
                
                tn, fp, fn, tp = cm.ravel()
                ll_score = calculate_ll_metric(tp, fp, fn, tn)
                accuracy = (tp + tn) / len(y_test_i)
                
                if ll_score >= threshold and accuracy >= threshold:
                    if hasattr(model_i, 'predict_proba'):
                        proba_full = model_i.predict_proba(X_full_model)[:, 1]
                    else:
                        if hasattr(model_i, 'decision_function'):
                            decision = model_i.decision_function(X_full_model)
                            proba_full = (decision - decision.min()) / (decision.max() - decision.min() + 1e-10)
                        else:
                            proba_full = model_i.predict(X_full_model).astype(float)
                    
                    iteration_predictions.append(proba_full)
                    iteration_weights.append(ll_score)
                    valid_count += 1
            
            except:
                continue
        
        if valid_count < 50:
            print(f"  Threshold {threshold:.2f}: Insufficient valid classifiers ({valid_count})")
            continue
        
        iteration_predictions = np.array(iteration_predictions)
        iteration_weights = np.array(iteration_weights)
        normalized_weights = iteration_weights / iteration_weights.sum()
        
        final_pred_weighted = np.average(iteration_predictions, axis=0, weights=normalized_weights)
        final_pred_unweighted = np.mean(iteration_predictions, axis=0)
        
        threshold_sensitivity_results[model_name].append({
            'threshold': threshold,
            'valid_classifiers': valid_count,
            'mean_ll_score': np.mean(iteration_weights),
            'weighted_pred': final_pred_weighted,
            'unweighted_pred': final_pred_unweighted
        })
        
        print(f"  Threshold {threshold:.2f}: {valid_count}/{n_iterations} valid ({100*valid_count/n_iterations:.1f}%)")
        print(f"    Mean LL: {np.mean(iteration_weights):.4f}")

    best_threshold_idx = np.argmax([r['valid_classifiers'] for r in threshold_sensitivity_results[model_name]])
    best_result = threshold_sensitivity_results[model_name][best_threshold_idx]
    
    final_predictions_by_threshold[model_name] = {
        'threshold': best_result['threshold'],
        'weighted': best_result['weighted_pred'],
        'unweighted': best_result['unweighted_pred'],
        'valid_count': best_result['valid_classifiers']
    }
    
    print(f"\n  Selected threshold: {best_result['threshold']:.2f} ({best_result['valid_classifiers']} classifiers)")

print("PU BAGGING COMPLETE")

Iterations per model: 500
PU BAGGING: RandomForest
  Threshold 0.55: 500/500 valid (100.0%)
    Mean LL: 0.8233
  Threshold 0.60: 500/500 valid (100.0%)
    Mean LL: 0.8233
  Threshold 0.65: 500/500 valid (100.0%)
    Mean LL: 0.8233
  Threshold 0.70: 500/500 valid (100.0%)
    Mean LL: 0.8233

  Selected threshold: 0.55 (500 classifiers)
PU BAGGING: XGBoost
  Threshold 0.55: 500/500 valid (100.0%)
    Mean LL: 0.8232
  Threshold 0.60: 500/500 valid (100.0%)
    Mean LL: 0.8232
  Threshold 0.65: 500/500 valid (100.0%)
    Mean LL: 0.8232
  Threshold 0.70: 500/500 valid (100.0%)
    Mean LL: 0.8232

  Selected threshold: 0.55 (500 classifiers)
PU BAGGING: LightGBM
  Threshold 0.55: 500/500 valid (100.0%)
    Mean LL: 0.8161
  Threshold 0.60: 500/500 valid (100.0%)
    Mean LL: 0.8161
  Threshold 0.65: 500/500 valid (100.0%)
    Mean LL: 0.8161
  Threshold 0.70: 500/500 valid (100.0%)
    Mean LL: 0.8161

  Selected threshold: 0.55 (500 classifiers)
PU BAGGING: LogisticReg
  Threshold 0.

In [20]:
# STEP 13: Spatial CV

cv_fold_results = defaultdict(lambda: {'auc': [], 'f1': [], 'recall': [], 'precision': []})

n_splits_cv = 5
gkf = GroupKFold(n_splits=n_splits_cv)

for model_name in best_models.keys():    
    is_tree_model = model_name in best_tree_models
    X_cv = X_tree_balanced if is_tree_model else X_linear_balanced_filtered
    
    for fold, (train_idx, test_idx) in enumerate(gkf.split(X_cv, y_balanced, groups=groups_balanced), 1):
        X_tr, X_te = X_cv[train_idx], X_cv[test_idx]
        y_tr, y_te = y_balanced[train_idx], y_balanced[test_idx]
        
        model = clone(best_models[model_name])
        model.fit(X_tr, y_tr)
        
        y_pred = model.predict(X_te)
        
        if hasattr(model, 'predict_proba'):
            y_proba = model.predict_proba(X_te)[:, 1]
        else:
            if hasattr(model, 'decision_function'):
                df = model.decision_function(X_te)
                y_proba = (df - df.min()) / (df.max() - df.min() + 1e-10)
            else:
                y_proba = y_pred.astype(float)
        
        cv_fold_results[model_name]['auc'].append(roc_auc_score(y_te, y_proba))
        cv_fold_results[model_name]['f1'].append(f1_score(y_te, y_pred, zero_division=0))
        cv_fold_results[model_name]['recall'].append(recall_score(y_te, y_pred, zero_division=0))
        cv_fold_results[model_name]['precision'].append(precision_score(y_te, y_pred, zero_division=0))

spatial_cv_summary = []
for model_name in best_models.keys():
    auc_scores = cv_fold_results[model_name]['auc']
    f1_scores = cv_fold_results[model_name]['f1']
    recall_scores = cv_fold_results[model_name]['recall']
    precision_scores = cv_fold_results[model_name]['precision']
    
    spatial_cv_summary.append({
        'Model': model_name,
        'AUC_Mean': np.mean(auc_scores),
        'AUC_CI_Low': np.percentile(auc_scores, 2.5),
        'AUC_CI_High': np.percentile(auc_scores, 97.5),
        'F1_Mean': np.mean(f1_scores),
        'F1_CI_Low': np.percentile(f1_scores, 2.5),
        'F1_CI_High': np.percentile(f1_scores, 97.5),
        'Recall_Mean': np.mean(recall_scores),
        'Precision_Mean': np.mean(precision_scores)
    })
    
spatial_cv_df = pd.DataFrame(spatial_cv_summary)
spatial_cv_df = spatial_cv_df.sort_values('AUC_Mean', ascending=False)

print("SPATIAL CV SUMMARY (95% CI from empirical fold scores)")
print(spatial_cv_df[['Model', 'AUC_Mean', 'AUC_CI_Low', 'AUC_CI_High', 'F1_Mean']].round(3).to_string(index=False))

spatial_cv_df.to_csv('spatial_cv.csv', index=False)

SPATIAL CV SUMMARY (95% CI from empirical fold scores)
       Model  AUC_Mean  AUC_CI_Low  AUC_CI_High  F1_Mean
RandomForest     0.883       0.841        0.918    0.820
         SVM     0.878       0.852        0.911    0.788
     XGBoost     0.876       0.821        0.920    0.811
 LogisticReg     0.875       0.860        0.894    0.797
         MLP     0.874       0.849        0.897    0.794
    LightGBM     0.867       0.831        0.903    0.762
         KNN     0.862       0.844        0.891    0.768


In [26]:
# STEP 14: Model selection across differrent Weighting Scheme

weighting_schemes = [0.4, 0.5, 0.6, 0.7]
model_selection_sensitivity = []

for f1_weight in weighting_schemes:
    auc_weight = 1 - f1_weight
    
    for _, row in spatial_cv_df.iterrows():
        rank_score = f1_weight * row['F1_Mean'] + auc_weight * row['AUC_Mean']
        model_selection_sensitivity.append({
            'Model': row['Model'],
            'F1_Weight': f1_weight,
            'AUC_Weight': auc_weight,
            'Rank_Score': rank_score
        })

selection_df = pd.DataFrame(model_selection_sensitivity)
print("\n Best Model across weighting schemes:")
for f1_weight in weighting_schemes:
    subset = selection_df[selection_df['F1_Weight'] == f1_weight].sort_values('Rank_Score', ascending=False)
    best = subset.iloc[0]['Model']
    print(f"  F1={f1_weight:.1f}, AUC={1-f1_weight:.1f}: {best}")

best_model = selection_df.groupby('Model')['Rank_Score'].mean().idxmax()
print(f"\Best model: {best_model}")

final_model_name = best_model
final_predictions_weighted = final_predictions_by_threshold[final_model_name]['weighted']
final_predictions_unweighted = final_predictions_by_threshold[final_model_name]['unweighted']

print(f"Using weighted ensemble for {final_model_name}")


 Best Model across weighting schemes:
  F1=0.4, AUC=0.6: RandomForest
  F1=0.5, AUC=0.5: RandomForest
  F1=0.6, AUC=0.4: RandomForest
  F1=0.7, AUC=0.3: RandomForest
\Best model: RandomForest
Using weighted ensemble for RandomForest


In [36]:
# STEP 15: Probability Calibration & Distribution

X_test_holdout = np.vstack([
    X_tree_full[P_test_indices] if final_model_name in best_tree_models else X_linear_full[P_test_indices],
    X_tree_full[U_test_indices] if final_model_name in best_tree_models else X_linear_full[U_test_indices]
])

y_test_holdout = np.hstack([np.ones(n_P_test), np.zeros(len(U_test_indices))])

test_indices_combined = P_test_indices + U_test_indices
final_pred_test = final_predictions_weighted[test_indices_combined]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

prob_true, prob_pred = calibration_curve(y_test_holdout, final_pred_test, n_bins=10, strategy='uniform')
ax1.plot(prob_pred, prob_true, marker='o', linewidth=2, label=final_model_name)
ax1.plot([0, 1], [0, 1], 'k--', label='Perfect calibration')
ax1.set_xlabel('Predicted probability', fontsize=12)
ax1.set_ylabel('True probability (in bin)', fontsize=12)
ax1.set_title('Calibration Curve (Test Set)', fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(alpha=0.3)

ax2.hist(final_pred_test[y_test_holdout == 1], bins=20, alpha=0.6, label='Positives', color='red')
ax2.hist(final_pred_test[y_test_holdout == 0], bins=20, alpha=0.6, label='Unlabeled', color='blue')
ax2.set_xlabel('Predicted probability', fontsize=12)
ax2.set_ylabel('Frequency', fontsize=12)
ax2.set_title('Prediction Distribution', fontsize=14, fontweight='bold')
ax2.legend()
ax2.grid(alpha=0.3)

ax1.text(0.02, 0.95, '(a)', transform=ax1.transAxes,
         fontsize=14, fontweight='bold', va='top')

ax2.text(0.02, 0.95, '(b)', transform=ax2.transAxes,
         fontsize=14, fontweight='bold', va='top')

plt.tight_layout()

plt.rcParams.update({
    'font.family': 'serif',
    'font.serif': ['Times New Roman'],
    'axes.titlesize': 14,
    'axes.labelsize': 13,
    'xtick.labelsize': 12,
    'ytick.labelsize': 12,
    'legend.fontsize': 12
})

plt.savefig('probability_calibration_and_distribution.png', dpi=1200, bbox_inches='tight')
plt.close()

In [40]:
# STEP 16: Holdout test set evaluation

test_auc = roc_auc_score(y_test_holdout, final_pred_test)
test_pred_binary = (final_pred_test > 0.5).astype(int)

recall_on_known_positives = recall_score(y_test_holdout, test_pred_binary)
precision_apparent = precision_score(y_test_holdout, test_pred_binary, zero_division=0)
f1_apparent = f1_score(y_test_holdout, test_pred_binary, zero_division=0)

print(f"\nTest set performance ({final_model_name}):")
print(f"  AUC (lower bound): {test_auc:.4f}")
print(f"  Recall on known positives: {recall_on_known_positives:.4f}")
print(f"  Apparent precision (biased by U contamination): {precision_apparent:.4f}")
print(f"  Apparent F1 (biased by U contamination): {f1_apparent:.4f}")
print(f"\nWARNING: Precision/F1 are underestimates due to unlabeled positives in test negatives")


Test set performance (RandomForest):
  AUC (lower bound): 0.8834
  Recall on known positives: 0.8447
  Apparent precision (biased by U contamination): 0.7982
  Apparent F1 (biased by U contamination): 0.8208



In [32]:
# STEP 17: Final Outputs
data['dump_risk_prob'] = final_predictions_weighted

predictions_output = data[['centroid_x', 'centroid_y', 'dump_risk_prob']].copy()
predictions_output.columns = ['Longitude', 'Latitude', 'Risk_Probability']
predictions_output.to_csv('final_dump_risk_predictions.csv', index=False)

model_comparison = spatial_cv_df.copy()
model_comparison['PU_Valid_Classifiers'] = [
    final_predictions_by_threshold[m]['valid_count'] if m in final_predictions_by_threshold else 0
    for m in model_comparison['Model']
]
model_comparison['PU_Threshold'] = [
    final_predictions_by_threshold[m]['threshold'] if m in final_predictions_by_threshold else np.nan
    for m in model_comparison['Model']
]
model_comparison.to_csv('model_comparison_final.csv', index=False)

In [35]:
# STEP 18: Feature Importance
final_model = best_models[final_model_name]

feature_name_mapping = {
    'population_density': 'Population Density',
    'poverty_index': 'Poverty Index',
    'infrastructure_index': 'Infrastructure Index',
    'livelihood_index': 'Livelihood Index',
    'sts': 'STS',
    'informal_settlement': 'Informal Settlement',
    'road_intersection': 'Road Intersection',
    'railine': 'Railway',
    'road': 'Road',
    'drainage': 'Drainage',
    'water': 'Waterbody',
    'buildings': 'Buildings',
    'lst': 'LST',
    'dem': 'DEM',
    'wgr': 'Waste Generation Rate'
}


importance_df = pd.DataFrame({
    'Feature_Short': feature_names_short,
    'Feature': [feature_name_mapping.get(f, f) for f in feature_names_short],
    'Importance': feature_importance,
    'Percentage': feature_importance * 100
}).sort_values('Importance', ascending=False)

importance_df['Cumulative_Percentage'] = importance_df['Percentage'].cumsum()

plt.rcParams.update({
    'font.family': 'serif',
    'font.serif': ['Times New Roman'],
    'font.size': 14,
    'axes.labelsize': 18,
    'axes.titlesize': 20,
    'xtick.labelsize': 16,
    'ytick.labelsize': 16,
    'legend.fontsize': 14,
    'figure.dpi': 1200
})


fig, ax1 = plt.subplots(figsize=(14, 8))
x_pos = np.arange(len(importance_df))


bars = ax1.bar(
    x_pos,
    importance_df['Percentage'],
    color='#d9d9d9',          # ash gray
    edgecolor='black',
    linewidth=0.8,
    width=0.7,
    alpha=0.95,
    label='Individual Importance'
)


for idx, pct in zip(x_pos, importance_df['Percentage']):
    ax1.text(
        idx, pct / 2,
        f'{pct:.1f}%',
        ha='center',
        va='center',
        fontsize=12,
        fontweight='bold'
    )

ax1.set_xlabel('Predictors', fontweight='bold')
ax1.set_ylabel('Importance (%)', fontweight='bold')
ax1.set_ylim(0, importance_df['Percentage'].max() * 1.15)
ax1.grid(axis='y', linestyle='--', alpha=0.3)
ax1.set_axisbelow(True)
ax2 = ax1.twinx()
ax2.plot(
    x_pos,
    importance_df['Cumulative_Percentage'],
    color='#b22222',
    marker='o',
    linewidth=2.5,
    markersize=6,
    markerfacecolor='#b22222',
    markeredgecolor='black',
    label='Cumulative Importance'
)

for idx, cum in zip(x_pos, importance_df['Cumulative_Percentage']):
    ax2.text(
        idx, cum + 2,
        f'{cum:.1f}%',
        ha='center',
        va='bottom',
        fontsize=11,
        fontweight='bold',
        color='#b22222'
    )

ax2.set_ylabel('Cumulative Importance (%)', fontweight='bold')
ax2.set_ylim(0, 105)
ax2.axhline(80, color='#2ca25f', linestyle='--', linewidth=1.5, alpha=0.7, label='80% threshold')
ax2.axhline(95, color='#f16913', linestyle='--', linewidth=1.5, alpha=0.7, label='95% threshold')

ax1.set_xticks(x_pos)
ax1.set_xticklabels(
    importance_df['Feature'],
    rotation=90,
    ha='right'
)

legend1 = ax1.legend(
    loc='center left',
    bbox_to_anchor=(0.78, 0.55),
    framealpha=0.9
)

legend2 = ax2.legend(
    loc='center left',
    bbox_to_anchor=(0.78, 0.40),
    framealpha=0.9
)

ax1.add_artist(legend1)


plt.tight_layout()
plt.savefig(
    'pareto_chart.png',
    dpi=1200,
    bbox_inches='tight',
    facecolor='white'
)
plt.show()

print("✓ Saved: pareto.png")


✓ Saved: pareto.png


In [39]:
# STEP 19 : ROC

required_vars = [
    'final_predictions_by_threshold',
    'P_test_indices',
    'U_test_indices'
]

for v in required_vars:
    if v not in globals():
        raise ValueError(f"Required variable '{v}' not found.")

y_true = np.hstack([
    np.ones(len(P_test_indices)),
    np.zeros(len(U_test_indices))
])

test_indices = P_test_indices + U_test_indices

model_order = list(final_predictions_by_threshold.keys())
if "RandomForest" in model_order:
    model_order.remove("RandomForest")
    model_order.append("RandomForest")

plt.figure(figsize=(10, 6))

for model_name in model_order:

    result = final_predictions_by_threshold[model_name]
    y_score = result['weighted'][test_indices]
    fpr, tpr, _ = roc_curve(y_true, y_score)
    roc_auc = auc(fpr, tpr)
    fpr_unique, unique_idx = np.unique(fpr, return_index=True)
    tpr_unique = tpr[unique_idx]
    fpr_dense = np.linspace(0, 1, 2000)
    pchip = PchipInterpolator(fpr_unique, tpr_unique)
    tpr_smooth = pchip(fpr_dense)
    tpr_smooth = np.clip(tpr_smooth, 0, 1)
    tpr_smooth[0] = 0.0
    tpr_smooth[-1] = 1.0
    if model_name == "RandomForest":
        lw = 1
        zo = 10
    else:
        lw = 1
        zo = 2

    plt.plot(
        fpr_dense,
        tpr_smooth,
        linewidth=lw,
        zorder=zo,
        label=f"{model_name} (AUC = {roc_auc:.3f})"
    )

plt.plot(
    [0, 1], [0, 1],
    'k--',
    linewidth=1.2,
    label="Random",
    zorder=1
)

plt.xlabel("False Positive", fontweight='bold')
plt.ylabel("True Positive", fontweight='bold')

plt.legend(loc="lower right", fontsize=12)
plt.grid(alpha=0.3, linestyle='--')

plt.xlim(0, 1)
plt.ylim(0, 1)

plt.tight_layout()

plt.savefig(
    "ROC.png",
    dpi=1200,
    bbox_inches="tight"
)
plt.close()

print("✓ Saved: ROC.png")

✓ Saved: ROC.png
